In [1]:
import asyncio
from codecs import StreamReader
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent

from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from dotenv import load_dotenv
from autogen_agentchat.ui import Console
import os

In [2]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

### Round Robin Team 1 - searches Web and returns accurate results after User approves

In [3]:
# web search tool 
from langchain_community.utilities import GoogleSerperAPIWrapper

SERPER_API_KEY=os.environ['SERPER_API_KEY']

search_tool_wrapper = GoogleSerperAPIWrapper(type='search',serper_api_key=SERPER_API_KEY)

def search_web(query:str) ->str:
    """Search the web for the given query and return the results."""
    try:
        results = search_tool_wrapper.run(query)
        return results
    except Exception as e:
        print(f"Error occurred while searching the web: {e}")
        return "No results found." 

In [4]:
search_web("what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026?")

'The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... Missing: adequacy | Show results with:adequacy. Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type: ... Q9: What is changing for Plan Year 2026 regarding Network Adequacy data submissions, MPMS validations, and CMS review results? CMS is making the following ... This final rule includes a number of new policies that will improve these programs for contract year 2026, as well as codify existing Part C ... CMS proposes significant reforms to the prior authorization process, requiring plans to adopt electronic systems for streamlining requests, ... Key Changes for PY2026: NA Template: Ne

In [5]:
web_search_agent = AssistantAgent(
    name = 'WebSearchAgent',
    description= 'An agent for searching the web for information.',
    model_client=model_client,
    tools = [search_web],
    reflect_on_tool_use=True,
    system_message='''
        You are a web search agent.
        Your only tool is search_web - use it to find the information you need.

        You make only one search call at a time.
        
        Once you have the results, you never do summary of results.
    ''',
)

In [7]:
searchTeamUser_proxy_agent = UserProxyAgent(
    name ='SearchTeamUserProxy',
    description='you are a user proxy agent for web search team',
    input_func=input
)

In [8]:
termination_condition = TextMentionTermination(text='APPROVE')

In [9]:
webSearchTeam = RoundRobinGroupChat(
    participants=[web_search_agent, searchTeamUser_proxy_agent],
    termination_condition=termination_condition,
    max_turns=4
)

In [12]:
async def run_team():
    task = TextMessage(content='what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026?',source='user')
    stream=webSearchTeam.run_stream(task=task)
    await Console(stream)

await run_team()

---------- user ----------
what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026?
---------- WebSearchAgent ----------
[FunctionCall(id='call_s6enlzhwVqFxd9tqgdjfa6WK', arguments='{"query":"Key Changes in CMS Network adequacy requirements for Plan Year 2026"}', name='search_web')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Key Changes for PY2026: NA Template: New columns have been added to the NA ... If the file name is changed, CMS may issue a required correction. CMS proposes to codify the requirements established in the final part one and final part two guidance for 2026 and future years of the Medicare ... Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type: ... Key changes include a meaningful difference standard for plan options, EHB premium review, ECP reviews, no early bird deadline, and MPMS module changes. This final rul

### Round Robin Team 2 - summarizes the results in simple language and returns accurate results after User approves

In [13]:
editorAgent = AssistantAgent(
    name = 'editorAgent',
    model_client=model_client,
    description='Edit the text',
    system_message="You edit the summary provided to you to make it easy to understand. keep it under 50 words."
)

In [14]:
summaryTeamUser_proxy_agent = UserProxyAgent(
    name ='summaryTeamUserProxy',
    description='you are a user proxy agent for Summary team',
    input_func=input
)

In [15]:
summaryTeam = RoundRobinGroupChat(
    participants=[editorAgent, summaryTeamUser_proxy_agent],
    termination_condition=termination_condition,
    max_turns=4
)

### Selector Team that combines the 2 teams

In [26]:
from autogen_agentchat.agents import SocietyOfMindAgent

webSearchAgentSOM = SocietyOfMindAgent(name="society_of_mind_Web_Search", description="Society of Mind agent with inner team as web search team",team=webSearchTeam, model_client=model_client)
summaryAgentSOM = SocietyOfMindAgent("society_of_mind_Summary", description="Society of mind agent with inner team as summary team",team=summaryTeam, model_client=model_client)


In [27]:
planning_agent = AssistantAgent(
    name="PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        webSearchAgentSOM: Searches for information
        summaryAgentSOM: Summarizes the information into simple text

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, if summary agent gave good summary end with "TERMINATE".
    """,
)


In [28]:
planningUserProxyAgent = UserProxyAgent(
    name ='UserProxy',
    description='you are a user proxy agent for Planning team',
    input_func=input
)

In [20]:
from autogen_agentchat.conditions import TextMentionTermination,MaxMessageTermination

finalText_mention_termination = TextMentionTermination('TERMINATE')
max_message_termination = MaxMessageTermination(max_messages=10)
combined_termination = finalText_mention_termination | max_message_termination

In [21]:
selector_prompt = '''
Select an agent to perform the task.

{roles}

current conversation history :
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
Make sure that the planning agent has assigned task before other agents start working.
Only select one agent.
'''

In [29]:
from autogen_agentchat.teams import SelectorGroupChat
selector_team = SelectorGroupChat(
    participants=[planning_agent, webSearchAgentSOM, summaryAgentSOM, planningUserProxyAgent],
    model_client=model_client,
    termination_condition=combined_termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True)

In [35]:
task="what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 10 changes."

In [37]:
await selector_team.reset()

In [38]:
from autogen_agentchat.ui import Console

await Console(selector_team.run_stream(task=task))

---------- user ----------
what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 10 changes.
---------- PlanningAgent ----------
To provide a summary of the key changes in CMS Network adequacy requirements for Plan Year 2026, we need to first gather relevant and updated information on this topic. Here is the breakdown of tasks:

1. webSearchAgentSOM: Conduct a web search to identify and gather detailed information about the key changes in CMS Network adequacy requirements for Plan Year 2026.
2. summaryAgentSOM: Summarize the information into the top 10 changes, ensuring the summary is concise and clear.

After these tasks have been completed, we will review the summary for accuracy and completeness.
---------- WebSearchAgent ----------
[FunctionCall(id='call_8t2yP1GEc2j39nL75mQTUAXw', arguments='{"query":"CMS Network adequacy requirements changes Plan Year 2026"}', name='search_web')]
---------- WebSearchAgent ----------
[FunctionExecutionRe

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 10 changes.', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=151, completion_tokens=120), metadata={}, content='To provide a summary of the key changes in CMS Network adequacy requirements for Plan Year 2026, we need to first gather relevant and updated information on this topic. Here is the breakdown of tasks:\n\n1. webSearchAgentSOM: Conduct a web search to identify and gather detailed information about the key changes in CMS Network adequacy requirements for Plan Year 2026.\n2. summaryAgentSOM: Summarize the information into the top 10 changes, ensuring the summary is concise and clear.\n\nAfter these tasks have been completed, we will review the summary for accuracy and completeness.', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgen

TaskResult(messages=[
    TextMessage(source='user', models_usage=None, metadata={}, content='what are the Key Changes in CMS Network adequacy requirements for Plan Year 2026? Summarize in top 10 changes.', type='TextMessage'), 
    
    TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=151, completion_tokens=120), metadata={}, content='To provide a summary of the key changes in CMS Network adequacy requirements for Plan Year 2026, we need to first gather relevant and updated information on this topic. Here is the breakdown of tasks:\n\n1. webSearchAgentSOM: Conduct a web search to identify and gather detailed information about the key changes in CMS Network adequacy requirements for Plan Year 2026.\n2. summaryAgentSOM: Summarize the information into the top 10 changes, ensuring the summary is concise and clear.\n\nAfter these tasks have been completed, we will review the summary for accuracy and completeness.', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=262, completion_tokens=24), metadata={}, content=[FunctionCall(id='call_8t2yP1GEc2j39nL75mQTUAXw', arguments='{"query":"CMS Network adequacy requirements changes Plan Year 2026"}', name='search_web')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Issuers must only include providers on the NA Template that are not at known risk for potential contract termination or non-renewal for the upcoming plan year. Q9: What is changing for Plan Year 2026 regarding Network Adequacy data submissions, MPMS validations, and CMS review results? CMS is making the following ... The Contract Year (CY) 2026 MA and Part D final rule implements changes related to prescription drug coverage, the Medicare Prescription Payment Plan, dual ... This final rule includes a number of new policies that will improve these programs for contract year 2026, as well as codify existing Part C ... Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type ... CMS proposes significant reforms to the prior authorization process, requiring plans to adopt electronic systems for streamlining requests, ... Contracts that fail to meet network adequacy requirements during the contract year may be subject to compliance or enforcement actions. Page 8. Medicare ... Health plans that will submit applications for CY 2026 are required to meet all network adequacy requirements in addition to converting any ... This section describes how CMS will conduct reviews of the network adequacy standards for medical QHP and SADP certification for the 2026 plan ... Starting in 2026, plans sold in state-based exchanges will be required to meet time and distance standards for provider access that are already applied to ...', name='search_web', call_id='call_8t2yP1GEc2j39nL75mQTUAXw', is_error=False)], type='ToolCallExecutionEvent'), 
    
    TextMessage(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=550, completion_tokens=356), metadata={}, content='Here are some changes related to CMS Network Adequacy requirements for Plan Year 2026:\n\n1. **Updated Provider Listings**: Issuers must include only providers that do not have a known risk for contract termination or non-renewal for the upcoming plan year.\n\n2. **New Facility Specialty Types**: Introduction of new facility specialty types in network adequacy applications to ensure diverse provider types are available within the network.\n\n3. **Prior Authorization Reforms**: Significant reforms to the prior authorization process by requiring plans to adopt electronic systems, improving the efficiency of requests.\n\n4. **Time and Distance Standards**: Starting in 2026, state-based exchanges will require plans to meet specific time and distance standards for provider access.\n\n5. **Compliance and Enforcement**: Contracts that fail to meet network adequacy requirements may face compliance or enforcement actions.\n\n6. **Medical QHP and SADP Certification Reviews**: Detailed processes on how CMS will review network adequacy standards for Qualified Health Plans (QHP) and Stand-Alone Dental Plans (SADP).\n\n7. **Data Submission Changes**: Changes in how network adequacy data submissions and CMS review results are validated through MPMS (Medicare Plan Management System).\n\n8. **Integration with Prescription Coverage Rules**: Changes also pertain to how network adequacy aligns with prescription drug coverage rules for Contract Year 2026.\n\n9. **Standards for Dual Eligible Programs**: Codified standards that improve the programs for dually eligible Medicare beneficiaries, ensuring adequate provider networks.\n\n10. **Application Requirements**: Health plans submitting applications must meet all updated network adequacy requirements and adapt their networks accordingly. \n\nFor a complete understanding, please refer to the CMS published guidelines and documentation for Plan Year 2026. ', type='TextMessage'), UserInputRequestedEvent(source='SearchTeamUserProxy', models_usage=None, metadata={}, request_id='80682715-744a-4506-abf3-67fba8cc5689', content='', type='UserInputRequestedEvent'), 
    
    TextMessage(source='SearchTeamUserProxy', models_usage=None, metadata={}, content='tell me top 10 changes', type='TextMessage'), ToolCallRequestEvent(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=973, completion_tokens=87), metadata={}, content=[FunctionCall(id='call_6iDGaN4XrHjt4r01PkRdNRac', arguments='{"query": "Key Changes CMS Network Adequacy Plan Year 2026"}', name='search_web'), FunctionCall(id='call_FIrdCap1N2FezlmqfDV3kIfg', arguments='{"query": "CMS Network Adequacy 2026 summary"}', name='search_web'), FunctionCall(id='call_FOOPLrIhWR2lTf9oA8QHuZ47', arguments='{"query": "Top changes CMS Network Adequacy requirements 2026"}', name='search_web')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='WebSearchAgent', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='Key Changes for PY2026: NA Template: New columns have been added to the NA Template: Provider Grouping and Appointment Scheduling Provider Phone Number. Key changes include a meaningful difference standard for plan options, EHB premium review, ECP reviews, no early bird deadline, and MPMS module changes. Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type: ... The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. This final rule revises the Medicare Advantage (Part C), Medicare Prescription Drug Benefit (Part D), Medicare cost plan, and Programs of All-Inclusive Care ... The Proposed Rule aims to improve provider directories by mandating real-time updates and integrating the provider directory information into ... Plan Year 2026 brings notable changes, including new network adequacy time and distance standards and updated provider validations. The 240-page rule, published on December 10, would revise the named programs with respect to Star Ratings, marketing and communications, ... Starting in 2026, plans sold in state-based exchanges will be required to meet time and distance standards for provider access that are already applied to ... In the 2026 NBPP, CMS is proposing to change this by requiring issuers to establish a premium payment threshold, under which coverage cannot be ...', name='search_web', call_id='call_6iDGaN4XrHjt4r01PkRdNRac', is_error=False), FunctionExecutionResult(content='QHP issuers are required to maintain a network that is sufficient in number and types of providers, including providers that specialize in mental health and ... The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. Overview of Network Adequacy Changes for CY 2026 Applications. For the CY 2026 applications, CMS has introduced a few updates: New Facility Specialty Type ... This section describes how CMS will conduct reviews of the network adequacy standards for medical QHP and SADP certification for the 2026 plan ... CMS has implemented significant regulatory changes to network adequacy standards that require Medicare Advantage (MA) plans to quickly adapt to ensure ... SUMMARY: This final rule revises the Medicare Advantage (Part C), Medicare Prescription Drug Benefit (Part D), Medicare cost plan, ... On April 4, 2025, the Centers for Medicare & Medicaid Services (CMS) released the contract year (CY) 2026 final rule for the Medicare Advantage (MA) program. Starting in 2026, plans sold in state-based exchanges will be required to meet time and distance standards for provider access that are already applied to ... The Final Notice of Benefit and Payment Parameters for 2026 is out. Discover the key changes in network adequacy requirements. The 240-page rule, published on December 10, would revise the named programs with respect to Star Ratings, marketing and communications, health equity, drug ...', name='search_web', call_id='call_FIrdCap1N2FezlmqfDV3kIfg', is_error=False), FunctionExecutionResult(content='The Contract Year (CY) 2026 MA and Part D proposed rule aims to hold MA and Part D plans more accountable for delivering high-quality coverage. Missing: Top Adequacy This final rule includes a number of new policies that will improve these programs for contract year 2026, as well as codify existing Part C ... CMS finalized regulatory changes and added several new behavioral health specialties to network adequacy requirements. Regulatory changes ... New for PY2026 is the adoption of Alternative Time and Distance Standards for certain county types/specialties where CMS has determined that the base standards ... On April 4, 2025, the Centers for Medicare & Medicaid Services (CMS) released the contract year (CY) 2026 final rule for the Medicare Advantage (MA) program. Proposed updates aim to align standards with the changing market, stakeholder needs, and regulatory requirements, to assist organizations in ... Starting in 2026, plans sold in state-based exchanges will be required to meet time and distance standards for provider access that are already applied to ... The proposed home health payment update percentage for CY 2026 is 2.4 percent. Additionally, this rule proposes the CY 2026 fixed dollar loss ( ... CMS has network adequacy standards for MA contracts that consist of minimum numbers of providers, maximum travel time and distance to providers, ...', name='search_web', call_id='call_FOOPLrIhWR2lTf9oA8QHuZ47', is_error=False)], type='ToolCallExecutionEvent'), 
    
    TextMessage(source='WebSearchAgent', models_usage=RequestUsage(prompt_tokens=1901, completion_tokens=356), metadata={}, content='Here are the top 10 changes in CMS Network Adequacy requirements for Plan Year 2026:\n\n1. **New Facility Specialty Types**: CMS has introduced new facility specialty types in network adequacy applications to ensure a variety of provider types within networks.\n\n2. **Provider Directory Updates**: Enhanced focus on real-time updates to provider directories and better integration of provider information.\n\n3. **Time and Distance Standards**: Starting in 2026, plans in state-based exchanges must meet specific time and distance standards for provider access, similar to those already in use.\n\n4. **Enhanced Compliance Measures**: Issuers must now list only providers not at risk for contract termination or non-renewal for the upcoming plan year on their Network Adequacy (NA) Template.\n\n5. **Behavioral Health Specialties**: Addition of several new behavioral health specialties to network adequacy requirements.\n\n6. **Electronic Prior Authorization**: Mandated electronic systems for prior authorization to streamline requests and improve process efficiency.\n\n7. **EHB Premium Review and ECP Reviews**: Enhanced reviews related to Essential Health Benefits (EHB) and Essential Community Providers (ECP) to ensure greater network adequacy.\n\n8. **Quality Focused Reforms**: Proposed updates are aimed at holding Medicare Advantage and Part D plans more accountable for high-quality coverage.\n\n9. **Alternative Standards**: Adoption of Alternative Time and Distance Standards for certain counties and specialties, allowing for flexibility where base standards are difficult to meet.\n\n10. **Regulatory Changes**: A broad set of regulatory changes is proposed to align standards with evolving market needs and stakeholder requirements.\n\nThese changes are reflected in the regulatory documents and intended to improve provider access, enhance compliance, and assure diverse and adequate provider networks for beneficiaries.', type='TextMessage'), UserInputRequestedEvent(source='SearchTeamUserProxy', models_usage=None, metadata={}, request_id='baa1649c-2c1c-4a9d-bd96-489de2ccfb4e', content='', type='UserInputRequestedEvent'), 
    
    TextMessage(source='SearchTeamUserProxy', models_usage=None, metadata={}, content='approve', type='TextMessage'), 
    
    TextMessage(source='society_of_mind_Web_Search', models_usage=RequestUsage(prompt_tokens=809, completion_tokens=254), metadata={}, content='Here are the top 10 changes in CMS Network Adequacy requirements for Plan Year 2026:\n\n1. **New Facility Specialty Types**: Introduction of new facility specialties to ensure a diverse network of providers.\n\n2. **Provider Directory Updates**: Improved integration and real-time updates for provider directories.\n\n3. **Time and Distance Standards**: Implementation of specific time and distance standards for state-based exchange plans.\n\n4. **Enhanced Compliance Measures**: Only providers with secure contracts will be listed to ensure network stability.\n\n5. **Behavioral Health Specialties**: Incorporation of new behavioral health specialties in network requirements.\n\n6. **Electronic Prior Authorization**: Required use of electronic systems to improve prior authorization processes.\n\n7. **EHB and ECP Reviews**: Greater scrutiny on Essential Health Benefits and Essential Community Providers to ensure network adequacy.\n\n8. **Quality Focused Reforms**: Holding Medicare Advantage and Part D plans accountable for providing high-quality coverage.\n\n9. **Alternative Standards**: Flexible time and distance standards in areas where base standards are challenging to meet.\n\n10. **Regulatory Changes**: Comprehensive updates to align with evolving needs and enhance network adequacy.\n\nThese changes aim to improve access to care and ensure diverse and adequate provider networks.', type='TextMessage'), 
    
    TextMessage(source='editorAgent', models_usage=RequestUsage(prompt_tokens=453, completion_tokens=60), metadata={}, content='For CMS Network Adequacy 2026, changes include: new facility types, updated provider directories, specific distance standards, compliance measures, new behavioral health roles, electronic authorizations, Essential Health Benefits reviews, quality reforms, flexible standards, and comprehensive regulatory updates. These aim to enhance care access.', type='TextMessage'), UserInputRequestedEvent(source='summaryTeamUserProxy', models_usage=None, metadata={}, request_id='325b4cd2-3c71-46bd-96e7-a857557e0e92', content='', type='UserInputRequestedEvent'), 
    
    TextMessage(source='summaryTeamUserProxy', models_usage=None, metadata={}, content='tell me top 3 changes', type='TextMessage'), 
    
    TextMessage(source='editorAgent', models_usage=RequestUsage(prompt_tokens=535, completion_tokens=47), metadata={}, content='The top three changes in CMS Network Adequacy for 2026 are: introduction of new facility specialty types, updated provider directory processes for real-time integration, and implementation of specific time and distance standards for better state-based plan access.', type='TextMessage'), UserInputRequestedEvent(source='summaryTeamUserProxy', models_usage=None, metadata={}, request_id='bb10a2e7-77e9-4e1b-9446-2ba7a52aa9bf', content='', type='UserInputRequestedEvent'), 
    
    TextMessage(source='summaryTeamUserProxy', models_usage=None, metadata={}, content='approve', type='TextMessage'), 
    
    TextMessage(source='society_of_mind_Summary', models_usage=RequestUsage(prompt_tokens=202, completion_tokens=47), metadata={}, content='The top three changes in CMS Network Adequacy for 2026 are: introduction of new facility specialty types, updated provider directory processes for real-time integration, and implementation of specific time and distance standards for better state-based plan access.', type='TextMessage')], stop_reason='Maximum number of messages 10 reached, current message count: 12')